In [3]:
import sys, os

In [4]:
def xmap_reader (xmap_fn):
    qDict = {}
    rDict = {}
    with open(xmap_fn) as f:
        for l in f:
            if l[0] != "#":
                #h XmapEntryID	QryContigID	RefContigID	QryStartPos	QryEndPos	RefStartPos	RefEndPos	Orientation	Confidence	HitEnum	QryLen	RefLen	LabelChannel	Alignment
                #f int        	int        	int        	float      	float    	float      	float    	string     	float     	string 	float 	float 	int         	string   
                ll = l.strip().split("\t")
                qry_id, ref_id = ll[1:3]
                alignlist = ll[-1][1:-1].split(")(")
                alignedtups = map(lambda x: tuple(x.split(",")), alignlist)
                qDict.setdefault(qry_id, {}).setdefault(ref_id, []).append(alignedtups)
                rDict.setdefault(ref_id, {}).setdefault(qry_id, []).append(alignedtups)
    return qDict, rDict
                

In [5]:
def cmap_reader (qmap_fn, hmap=False):
    qMapDict = {}
    with open(qmap_fn) as f:
        for l in f:
            if l[0] != "#":
                #h CMapId	ContigLength	NumSites	SiteID	LabelChannel	Position	StdDev	Coverage	Occurrence
                #f int	float	int	int	int	float	float	float	float
                ll = l.strip().split("\t")
                qry_id, site_id, position = ll[0], ll[3], ll[5]
                if hmap:
                    hmap_cols = ll[9:]
                    qMapDict.setdefault(qry_id, {})[site_id] = (position, hmap_cols)
                else:
                    qMapDict.setdefault(qry_id, {})[site_id] = position
    return qMapDict

In [11]:
def merge_maps_into_simple_table(qXDict, qHDict, rDict, out=sys.stdout):
    '''
    print a list with format:
    q_contig_id, q_site_id, q_pos, r_contig_id, r_site_id, r_pos, hap_cols...,
    '''
    print >>out, '#q_contig_id', 'q_site_id', 'q_pos', 'r_contig_id', 'r_site_id', 'r_pos','HapSite','HapDelta','SitePhaseConf','DeltaPhaseConf','HapSiteScore','HapDeltaScore'
    for qry_id in qXDict:
        for ref_id in qXDict[qry_id]:
            for alignedtups in qXDict[qry_id][ref_id]: 
                for alignedtup in alignedtups:
                    #print alignedtup
                    r_site_id, q_site_id = alignedtup
                    qpos, hmap_cols = qHDict[qry_id][q_site_id]
                    rpos            = rDict[ref_id][r_site_id]
                    print >>out, qry_id, q_site_id, qpos, ref_id, r_site_id, rpos, " ".join(hmap_cols)

In [1]:
%cd /hpc/users/neffr01/2work/alex_na24385_phasing/hmap_combine

/sc/orga/work/neffr01/alex_na24385_phasing/hmap_combine


In [7]:
qXDict, rXDict = xmap_reader("alex_concat_hmaps_aligned.xmap")
qDict = cmap_reader("alex_concat_hmaps_aligned_q.cmap")
rDict = cmap_reader("alex_concat_hmaps_aligned_r.cmap")
qHDict = cmap_reader("alex_concat_hmaps.hmap",hmap=True)


In [12]:
fout = open("alex_merged_maps.txt", 'w')
merge_maps_into_simple_table(qXDict, qHDict, rDict, fout)
fout.close()